<a href="https://colab.research.google.com/github/iliyashm/svoyak-net/blob/ilya/svoyak_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 13.0 MB/s 
     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 6.5 MB 56.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import re

from torch.utils.data import Dataset, DataLoader, RandomSampler
from tqdm.auto import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

import torch
import numpy as np
import pandas as pd

tqdm.pandas()

In [4]:
def choose_from_top(probs, n=5):
    """
    Randomly(from the given probability distribution) choose the next word from the top n words.
    """
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)


def generate_some_text(input_str, max_len=2048):
    """
    Make GPT-2 continue input_str with text up to text_len long.
    If model returns <EOS>-token, generation stops.
    With min_len one can set a desired minimal length of generated text, but the limit is not hard.
    If model persists, text may come out shorter.
    """

    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to("cuda")

    model.eval()
    with torch.no_grad():

        for i in range(max_len - len(cur_ids)):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=20) # Choose next word
            if next_token_id == 2:
                break    # Stop generation
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to("cuda") * next_token_id], dim = 1) # Add the last word

        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        return output_text

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2").cuda()

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

In [10]:
examples = [
    "Шли нахуй тех, кто тебя кидал, останься с теми кто с тобой страдал.",
    "У каждого человека бывает такой период в жизни, когда ему кажется, что он никому не нужен.",
    "Живи, братуха, и не думай, что кто-то про тебя забыл. Друзья не забывают друга, а кто забыл, тот не был им.",
    "Сука, если я терплю, не означает, что мне не больно.",
    "Может и не получится , но попробовать стоит всегда",
    "Красивая — это когда красивая, а не когда грудь вываливается и трусы видно",
    "Слушай всех, прислушивайся к немногим, решай сам.",
    "Не нужно косить, под гламурных куриц, парни любят нежных, парни любят умниц",
    "Иметь хороших друзей - одна из лучших ценностей жизни..",
    "Будь самим собой, имей свою точку зрения, умей постоять за себя и за своих близких и тебя будут уважать!",
    "Живите так, что бы дома гордились, девушки любили, пацаны ценили."
]

In [33]:
examples = [["Вокруг The Beatles", "10", "Пол Маккартни утверждает, что мелодия ЭТОЙ ПЕСНИ пришла к нему во сне, и проснувшись, он был уверен, что просто накануне где-то слышал мелодию, а не сочинил.", "Yesterday "],
            ["Вокруг The Beatles", "20", "Критику обложки своего экспериментального альбома ОН прокомментировал так: «Шум казался очевидным. Меньше из-за явной наготы, больше с непривлекательностью пары»", "Джон Леннон"],
            ["Вокруг The Beatles", "30", "ОН впервые услышал «Beatles» 9 ноября 1961 года, и хотя не исполнил ни одной ноты, пресса стала называть его «пятым битлом»", "Брайан Эпштейн"],
            ["Нидерландские художники", "10", "Положение тени от левой руки капитана Кока показывает, что время действия — не позже 14 часов. Так что короткое название данное в 19 веке ЕГО масштабному произведению, не соответствует истине.", "Харменс Ван Рейн"],
            ["Нидерландские художники", "20", "Искусствовед Веджвуд отмечал, что только Тициан из ЕГО предшественников и Ренуар из преемников могли сравниться с ним в изображении женских форм.", "Рубенс"],
            ["Нидерландские художники", "30", "Художник постарался написать ИХ побольше. Но расшифровали до настоящего времени не больше сотни.", "Нидерландские пословицы"]]

In [34]:
class GPTDataset(Dataset):
    def __init__(self, examples):
        #data = ["ITEM: " + example for example in examples]
        data = []
        for example in examples:
          data.append(example[0] + " " + example[1] + " " + example[2] + " " + example[3])
        self.examples = list(map(tokenizer.encode, data))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item] + [2])

In [40]:
EPOCHS = 5
LEARNING_RATE = 1e-5
WARMUP_STEPS = 1

In [41]:
dataset = GPTDataset(examples)
sampler = RandomSampler(dataset)
synopsis_loader = DataLoader(dataset, sampler=sampler)
dataset

In [42]:
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,
                                            num_training_steps=EPOCHS*len(dataset))

proc_seq_count = 0
batch_count = 0

for epoch in range(EPOCHS):

    sum_loss = []

    
    t = tqdm(enumerate(synopsis_loader), total=len(dataset), desc=f"Epoch {epoch}, loss: ")
    for idx, batch in t:

        model.train()

        batch = batch.to("cuda")
        outputs = model(batch, labels=batch)

        loss = outputs[0]
        loss.backward()
        sum_loss.append(loss.item())

        del outputs, loss, batch
        torch.cuda.empty_cache()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

        t.set_description(f"Epoch {epoch}, loss: {np.mean(sum_loss) :.2}")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch 0, loss:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1, loss:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 2, loss:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 3, loss:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 4, loss:   0%|          | 0/6 [00:00<?, ?it/s]

In [44]:
for i in range(3):
    print(generate_some_text("Вокруг Beatles: "))

Вокруг Beatles:  Пол Маккартни утверждает, что мелодия ЭТОЙ ПЕСНИ пришла к нему во сне, и проснувшись, он был уверен, что просто накануне где-то слышал мелодию, а не сочинил. Yesterday 
Вокруг Beatles: ese 10 Пол Маккартни утверждает, что мелодия ЭТОЙ ПЕСНИ пришла к нему во сне, и проснувшись, он был уверен, что просто накануне где-то слышал мелодию, а не сочинил. Yesterday 
Вокруг Beatles:  Beatles 10 Пол Маккартни утверждает, что мелодия ЭТОЙ ПЕСНИ пришла к нему во сне, и проснувшись, он был уверен, что просто накануне где-то слышал мелодию, а не сочинил. Yesterday 
